In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "lower_grade_glioma_and_glioblastoma"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/TCGA.csv"
out_gene_data_file = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/lower_grade_glioma_and_glioblastoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Review subdirectories to find one related to lower_grade_glioma_and_glioblastoma
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)
print(f"Available TCGA directories: {tcga_dirs}")

# Look for directories related to lower_grade_glioma_and_glioblastoma
relevant_dirs = []
for dir_name in tcga_dirs:
    dir_lower = dir_name.lower()
    if trait.lower().replace("_", " ") in dir_lower.replace("_", " "):
        relevant_dirs.append(dir_name)
    elif "gbmlgg" in dir_lower:  # Specific acronym for lower grade glioma and glioblastoma
        relevant_dirs.append(dir_name)
    elif ("glioma" in dir_lower and "glioblastoma" in dir_lower):
        relevant_dirs.append(dir_name)

print(f"Potential relevant directories for {trait}: {relevant_dirs}")

if not relevant_dirs:
    print(f"No directory specifically relevant to the trait: {trait}")
    
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
    print(f"Task marked as completed. {trait} is not directly represented in the TCGA dataset.")
else:
    # If we found a relevant directory, proceed with loading the data
    selected_dir = relevant_dirs[0]
    print(f"Selected directory for {trait}: {selected_dir}")
    
    # Get the full path to the directory
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Find clinical and genetic data files
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    print(f"Clinical data file: {clinical_file_path}")
    print(f"Genetic data file: {genetic_file_path}")
    
    # Step 3: Load the data files
    clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
    genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')
    
    # Step 4: Print column names of clinical data
    print("\nClinical data columns:")
    print(clinical_df.columns.tolist())
    
    # Check if both datasets are available
    is_gene_available = not genetic_df.empty
    is_trait_available = not clinical_df.empty
    
    # Initial validation
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Available TCGA directories: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)', 'TCGA_Endometrioid_Cancer_


Clinical data columns:
['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history', 'animal_insect_allergy_types', 'asthma_history', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'days_to_additional_surgery_locoregional_procedure', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_performance_status_assessment', 'easte

### Step 2: Find Candidate Demographic Features

In [3]:
# Step 1: Identify potential age and gender columns
candidate_age_cols = [
    'age_at_initial_pathologic_diagnosis', 
    'days_to_birth',
    'first_diagnosis_age_asth_ecz_hay_fev_mold_dust',
    'first_diagnosis_age_of_animal_insect_allergy',
    'first_diagnosis_age_of_food_allergy'
]

candidate_gender_cols = ['gender']

# Step 2: Extract and preview these columns from the clinical data
# First, get the file paths
cohort_dir = os.path.join(tcga_root_dir, "TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)")
clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)

# Load the clinical data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)

# Extract and preview age columns
age_preview = {}
for col in candidate_age_cols:
    if col in clinical_df.columns:
        age_preview[col] = clinical_df[col].head(5).tolist()

print("Age column previews:")
print(age_preview)

# Extract and preview gender columns
gender_preview = {}
for col in candidate_gender_cols:
    if col in clinical_df.columns:
        gender_preview[col] = clinical_df[col].head(5).tolist()

print("\nGender column previews:")
print(gender_preview)


Age column previews:
{'age_at_initial_pathologic_diagnosis': [44.0, 50.0, 59.0, 56.0, 40.0], 'days_to_birth': [-16179.0, -18341.0, -21617.0, -20516.0, -14806.0], 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan], 'first_diagnosis_age_of_animal_insect_allergy': [nan, nan, nan, nan, nan], 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan]}

Gender column previews:
{'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}


### Step 3: Select Demographic Features

In [4]:
# Examine the age column previews
age_col = None
gender_col = None

# Select age column
age_cols_preview = {
    'age_at_initial_pathologic_diagnosis': [44.0, 50.0, 59.0, 56.0, 40.0], 
    'days_to_birth': [-16179.0, -18341.0, -21617.0, -20516.0, -14806.0], 
    'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [None, None, None, None, None], 
    'first_diagnosis_age_of_animal_insect_allergy': [None, None, None, None, None], 
    'first_diagnosis_age_of_food_allergy': [None, None, None, None, None]
}

# Select gender column
gender_cols_preview = {
    'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']
}

# For age, we want a column with actual age values, not missing values or negative numbers (days)
# 'age_at_initial_pathologic_diagnosis' contains valid age values
if 'age_at_initial_pathologic_diagnosis' in age_cols_preview and not all(val is None for val in age_cols_preview['age_at_initial_pathologic_diagnosis']):
    age_col = 'age_at_initial_pathologic_diagnosis'

# For gender, we want a column with valid gender values (male/female)
if 'gender' in gender_cols_preview and not all(val is None for val in gender_cols_preview['gender']):
    gender_col = 'gender'

# Print the chosen columns
print(f"Selected age column: {age_col}")
print(f"Selected gender column: {gender_col}")


Selected age column: age_at_initial_pathologic_diagnosis
Selected gender column: gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Use the Lower Grade Glioma and Glioblastoma directory identified in Step 1
selected_dir = "TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)"  # Use the correct directory from Step 1
cohort_dir = os.path.join(tcga_root_dir, selected_dir)

# Get the file paths for clinical and genetic data
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)

# Load the data
clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)

# Extract standardized clinical features using the provided trait variable
clinical_features = tcga_select_clinical_features(
    clinical_df, 
    trait=trait,  # Using the provided trait variable
    age_col=age_col, 
    gender_col=gender_col
)

# Convert gender values from text (FEMALE/MALE) to numeric (0/1)
if 'Gender' in clinical_features.columns:
    clinical_features['Gender'] = clinical_features['Gender'].apply(tcga_convert_gender)

# Save the clinical data to out_clinical_data_file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Saved clinical data with {len(clinical_features)} samples")

# Step 2: Normalize gene symbols in gene expression data
# Transpose to get genes as rows
gene_df = genetic_df

# Normalize gene symbols using NCBI Gene database synonyms
normalized_gene_df = normalize_gene_symbols_in_index(gene_df)
print(f"After normalization: {len(normalized_gene_df)} genes remaining")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene expression data")

# Step 3: Link clinical and genetic data
# Merge clinical features with genetic expression data
linked_data = clinical_features.join(normalized_gene_df.T, how='inner')
print(f"Linked data shape: {linked_data.shape} (samples x features)")

# Step 4: Handle missing values
cleaned_data = handle_missing_values(linked_data, trait_col=trait)
print(f"After handling missing values, data shape: {cleaned_data.shape}")

# Step 5: Determine if trait or demographics are severely biased
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait=trait)

# Step 6: Validate data quality and save cohort information
note = "The dataset contains gene expression data along with clinical information for lower grade glioma and glioblastoma patients from TCGA."

# Validate the data and determine if it's usable
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=len(normalized_gene_df) > 0,
    is_trait_available=True,
    is_biased=trait_biased,  # Use the actual bias determination
    df=cleaned_data,
    note=note
)

# Step 7: Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Saved usable linked data to {out_data_file}")
else:
    print("Dataset was determined to be unusable and was not saved.")

Saved clinical data with 1148 samples


After normalization: 19848 genes remaining


Saved normalized gene expression data
Linked data shape: (702, 19851) (samples x features)


After handling missing values, data shape: (702, 19850)
For the feature 'lower_grade_glioma_and_glioblastoma', the least common label is '0' with 5 occurrences. This represents 0.71% of the dataset.
The distribution of the feature 'lower_grade_glioma_and_glioblastoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 59.0
Min: 14.0
Max: 89.0
The distribution of the feature 'Age' in this dataset is fine.



Saved usable linked data to ../../output/preprocess/lower_grade_glioma_and_glioblastoma/TCGA.csv
